# Задание 1

[Свертка списка](https://ru.wikipedia.org/wiki/%D0%A1%D0%B2%D1%91%D1%80%D1%82%D0%BA%D0%B0_%D1%81%D0%BF%D0%B8%D1%81%D0%BA%D0%B0) -  это обобщенная операция над списком, c помощью которой можно преобразовать список в единое значение. Например, рассмотрим реализации свертки слева и свертки справа (левоассоциативную свертку и правоассоциативную свертку):

In [1]:
def foldl(f, x0, lst):
    if not lst:
        return x0
    return foldl(f, f(x0, lst[0]), lst[1:])

def foldr(f, x0, lst):
    if not lst:
        return x0
    return f(lst[0], foldr(f, x0, lst[1:]))

In [2]:
#сумма
foldl(lambda x, y: x + y, 0, [1, 2, 3])

6

In [3]:
#конкатенация
foldl(lambda x, y: '{}{}'.format(x, y), '', [1, 2, 3])

'123'

In [4]:
f = lambda x, y: x / y
foldl(f, 1, [1, 2, 3]), \
foldr(f, 1, [1, 2, 3])

(0.16666666666666666, 1.5)

Задача: реализовать foldl через foldr и наоборот. Вместо многоточий нужно вставить выражения, которые бы привели к нужному результату. Модифицировать сам список нельзя. 

In [5]:
def foldl2(f, x0, lst):
    return foldr(lambda l, g : (lambda x: g(f(x, l))), lambda x: x, lst)(x0)

In [6]:
foldl2(f, 1, [1, 2, 3])

0.16666666666666666

In [7]:
def foldr2(f, x0, lst):
    return foldr(lambda l, g : (lambda x: g(f(l, x))), lambda x: x, lst)(x0)

In [8]:
foldr2(f, 1, [1, 2, 3])

1.5

# Задание 2

нужно написать функцию, которая принимает две строки и проверяет, входит ли хотя бы одна перестановка второй строки в первую. Например:

> a = 'abcrotm'
> 
> b = 'tro'

функция def check_inv(a, b) вернет True, так как 'rot' содержится в 'abcrotm'. Нужно подумать как можно более оптимальный алгоритм и оценить его сложность. 

In [22]:
from collections import Counter
def check_letter(s, f, count_a, count_b):
    if count_a == count_b:
        s += 1
    if count_a != count_b and f:
        s -= 1

    return s

def step_up(c, s, a, b, d_a, d_b):
    char_a = a[c]
    char_b = b[c]
    if (char_a in b) and (char_b in a):
        f_a = (d_a[char_a] == d_b[char_a] and d_a[char_a] > 0)
        f_b = (d_a[char_b] == d_b[char_b] and d_a[char_b] > 0)
        d_a[char_a] += 1
        s = check_letter(s, f_a, d_a[char_a], d_b[char_a])
        d_b[char_b] += 1
        s = check_letter(s, f_b, d_a[char_b], d_b[char_b])
    else:
        d_a[char_a] += 1
        d_b[char_b] += 1
    return s, d_a, d_b

def step_a(c, s, a, d_a, d_b, step=1):
    char_a = a[c]
    if (char_a in d_b):
        f_a = (d_a[char_a] == d_b[char_a] and d_a[char_a] > 0)
        d_a[char_a] += step
        s = check_letter(s, f_a, d_a[char_a], d_b[char_a])
    else:
        d_a[char_a] += step
    return s, d_a, d_b

def check_inv(a, b):
    if len(b) >= len(a):
        return False
    d_a = {char:0 for char in set(a)}
    d_b = {char:0 for char in set(b)}
    i, j = 0, len(b) - 1
    s = 0
    for c in range(len(b)):
        s, d_a, d_b = step_up(c, s, a, b, d_a, d_b)

    if s == len(b):
        return True
    while j < len(a)-1:
        i += 1
        s, d_a, d_b = step_a(i-1, s, a, d_a, d_b, -1)
        j += 1
        s, d_a, d_b = step_a(j, s, a, d_a, d_b, 1)
        if s == len(b):
            return True
    return False

check_inv('abcrotm', 'ttro'), check_inv('abcrotm', 'tro')

(False, True)

Создание словаря - $O(len(a))+O(len(b)) = 2 * O(len(b))$ (т.к. $len(b) >= len(a)$), затем проxод окном в цикле по строке а в $O(len(a)) со сравнениями и обращением к элементу словаря(есть проверка на наличие, но словарь ограничен размером алфавита, так что он констатен). В общем, сложность линейная O(len(a)) и зависит от длины первой строки.

# Задание 3

Реализовать бинарное дерево (класс Tree), в нём методы __repr__, __str__, __iter__ (итерация только по листьям).

In [10]:
class Tree:
    def __init__(self, value=None, left=None, right=None):
        self.left = left
        self.right = right
        self.value = value
        
    def __iter__(self):
        if self.left == None and self.right == None:
            yield self.value
        if self.left != None:
            for node in self.left:
                yield node
        if self.right != None:
            for node in self.right:
                yield node

    def __str__(self):
        return str(self.value)
    
    def __repr__(self):
        return 'Tree({v}, {l}, {r})'.format(v = self.value, l = repr(self.left), r = repr(self.right))

tree = Tree(0, Tree(1, Tree(3), Tree(4)),                             
               Tree(2))

list(tree) == [3, 4, 2], list(tree), tree

(True,
 [3, 4, 2],
 Tree(0, Tree(1, Tree(3, None, None), Tree(4, None, None)), Tree(2, None, None)))

# Задание 4

Реализовать простейший калькулятор математических выражений:
- только целые числа
- **+**, **\-**, **\***, **\**
- скобки

**Можно использовать регулярные выражения**

In [27]:
import operator

def calc(expr):
    expr = expr.replace('(', '( ').replace(')', ' )').split()
    nums = []
    ops = []
    i = 0
    priority = {'-':0, '+':0, '*':1, '/':1, '(':0, ')':0}
    for t in expr:
        if t.isalnum():
            nums.append(t)
        elif t in ['+', '-', '*', '/']:
            while len(ops) > 0 and ops[-1] != '(' and (priority[t] < priority[ops[-1]] or priority[t] == priority[ops[-1]]):
                nums.append(ops.pop())
            ops.append(t)
        elif t == '(':
            ops.append(t)
        elif t == ')':
            while ops[-1] != '(':
                nums.append(ops.pop())
            if ops[-1] == '(':
                ops.pop()
    nums.extend(ops)
    return compute(nums)

def compute(rpn):
    ops = {'-': operator.sub, '+': operator.add, '*': operator.mul, '/':operator.truediv}
    i = 0
    s = []
    for i in l:
        if i.isalnum():
            s.append(i)
        else:
            s1 = int(s.pop())
            s2 = int(s.pop())
            op = ops[i]
            s.append(op(s2, s1))
    return s.pop()


calc('2 * (15 - 3 * 4) - 2') == 4

(True, 4)